In [1]:
import pandas as pd
import pyodbc
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from sqlalchemy import create_engine
import urllib
import sqlalchemy
import numpy as np
import datetime as dt
import os
import psycopg2 as pg
from psycopg2 import OperationalError
import csv 
import tkinter as tk
from tkinter import filedialog

In [2]:
server = 'ADMIN-PC'
database = 'Financial_Statement_Analysis'

quoted = urllib.parse.quote_plus('Driver={SQL Server};'
                                 'Server='+server+';'
                                 'Database='+database+';')
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server='+server+';'
                      'Database='+database+';'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

In [ ]:
path = r'D:\Tin hoc\Project\Báo cáo tài chính Power BI\BCTC'
files = []

root = tk.Tk()
root.geometry("1x1")

excels = filedialog.askopenfilenames(initialdir=path,
                            title="Choose A Report",
                            filetypes=(("Excel Files", "*.xlsx"),("All Files", "*.*") ))

for excel in excels: 
    excel = excel.replace('D:/Tin hoc/Project/Báo cáo tài chính Power BI/BCTC/', "")
    files.append(excel)

root.destroy()

df_new = None
lst_balance_sheet = []

def dim_balance_sheet(stock_id):   
    skiprows = [i for i in range(1,10)]
    list_bctc = []
    for index, file in enumerate(files):
        df_full = pd.read_excel(os.path.join(path, file), skiprows=lambda x: x in skiprows, sheet_name = None)
        bcdkt = list(df_full.keys())[0]
        df = df_full[bcdkt].iloc[:126]
        column_name = list(df.iloc[0].values)
        name = files[index][:3]
        column_name = [name] + column_name[1:]
        df.columns = column_name
        df = df.iloc[1:]
        df[name].iloc[95] = "     Cổ phiếu ưu đãi_ndh"
        df[name].iloc[104] = "          Cổ phiếu ưu đãi_vcsh"
        list_bctc.append(df)

    stock_name = [[j[:3],i] for i,j in enumerate(files)]
    stock_dict = {key: value for key, value in stock_name}
    num = stock_dict.get(stock_id)
    df = list_bctc[num]
    stock = df.columns[0]

    global df_new
    df_new = df.melt(id_vars=stock, var_name="Year", value_name="Value")
    df_new = df_new.rename(columns = {stock: "Item"})

    df_new_ts = df_new[df_new["Year"] == "2024"].copy()
    df_new_ts["Item"] = df_new_ts["Item"].str.strip()
    df_new_ts = df_new_ts.set_index(keys="Item")

    df_new_tsnh = df_new_ts.loc["TÀI SẢN NGẮN HẠN" : "TÀI SẢN DÀI HẠN"]
    lst_tsnh = list(df_new_tsnh.index)
    lst_tsnh = lst_tsnh[1:-1]

    df_new_tsdh = df_new_ts.loc["TÀI SẢN DÀI HẠN" : "TỔNG CỘNG TÀI SẢN"]
    lst_tsdh = list(df_new_tsdh.index)
    lst_tsdh = lst_tsdh[1:-1]

    df_new_npt = df_new_ts.loc["NỢ PHẢI TRẢ" : "VỐN CHỦ SỞ HỮU"]
    lst_npt = list(df_new_npt.index)
    lst_npt = lst_npt[1:-1]

    df_new_vcsh = df_new_ts.loc["VỐN CHỦ SỞ HỮU":]
    lst_vcsh= list(df_new_vcsh.index)
    lst_vcsh = lst_vcsh[1:-1]

    def phan_loai_lv1(row):
        item = row.loc["Item"]

        if item in lst_tsnh:
            return "A - TÀI SẢN NGẮN HẠN"
        
        elif item in lst_tsdh:
            return "B - TÀI SẢN DÀI HẠN"
        
        elif item in lst_npt:
            return "C - NỢ PHẢI TRẢ"
        
        elif item in lst_vcsh:
            return "D - VỐN CHỦ SỞ HỮU"

        else: return

    balance_sheet = [i.strip() for i in df_new["Item"].unique()]
    balance_sheet = pd.DataFrame(balance_sheet, columns=["Item"])
    balance_sheet["Lv2"] = balance_sheet.apply(phan_loai_lv1, axis="columns")
    balance_sheet['Lv1']= balance_sheet.apply(lambda row: "TỔNG CỘNG TÀI SẢN (270 = 100 + 200)" if row['Lv2'] in ["A - TÀI SẢN NGẮN HẠN", "B - TÀI SẢN DÀI HẠN"]
                                                else ("TỔNG CỘNG NGUỒN VỐN (440 = 300 + 400)" if row['Lv2'] in ["C - NỢ PHẢI TRẢ", "D - VỐN CHỦ SỞ HỮU"] else None),axis=1)
    lst_danh_dau = ["Tiền và tương đương tiền", "Giá trị thuần đầu tư ngắn hạn", "Các khoản phải thu", "Hàng tồn kho, ròng", "Tài sản lưu động khác",
                    "Phải thu dài hạn", "Tài sản cố định", "Giá trị ròng tài sản đầu tư", "Tài sản dở dang dài hạn", "Đầu tư dài hạn", "Tài sản dài hạn khác",
                    "Nợ ngắn hạn", "Nợ dài hạn"]
    balance_sheet["danh_dau"] = balance_sheet.apply(lambda row:1 if row['Item'] in lst_danh_dau else 0, axis=1)
    balance_sheet = balance_sheet[balance_sheet["danh_dau"] == 0]
    balance_sheet = balance_sheet.drop(columns="danh_dau")
    balance_sheet

    lst_tien_tuongduong_tien1 = ["Tiền-111", "Các khoản tương đương tiền-112"]
    lst_giatrithuan_dautu_nganhan1 = ["Đầu tư ngắn hạn-121", "Dự phòng đầu tư ngắn hạn-122", 
                                    "Đầu tư nắm giữ đến ngày đáo hạn-123"]
    lst_cackhoan_phaithu1 = ["Phải thu khách hàng-13101", "Trả trước người bán-132", "Phải thu nội bộ-133",
                            "Phải thu về XDCB-134", "Phải thu về cho vay ngắn hạn-135", "Phải thu khác-136",
                            "Dự phòng nợ khó đòi-137", "Tài sản thiếu chờ xử lý-139"]
    lst_hangtongkho_rong1 = ["Hàng tồn kho-141", "Dự phòng giảm giá HTK-149"]
    lst_taisan_luudong_khac1 = ["Chi phí trả trước ngắn hạn-151", "Thuế GTGT được khấu trừ-152", 
                            "Phải thu thuế khác-153", "Giao dịch mua bán lại trái phiếu chính phủ-154",
                            "Tài sản lưu động khác-155"]
    lst_phaithu_daihan1 = ["Phải thu khách hàng dài hạn-1310", "Trả trước người bán dài hạn-1311",
                        "Vốn kinh doanh ở các đơn vị trực thuộc-1312", "Phải thu nội bộ dài hạn-1313",
                        "Phải thu về cho vay dài hạn-1314", "Phải thu dài hạn khác-1315", 
                        "Dự phòng phải thu dài hạn-1316"]
    lst_taisan_codinh1 = ["GTCL TSCĐ hữu hình-210", "Nguyên giá TSCĐ hữu hình-211", 
                        "Khấu hao lũy kế TSCĐ hữu hình-212", "GTCL Tài sản thuê tài chính-223", 
                        "Nguyên giá tài sản thuê tài chính-224", "Khấu hao lũy kế tài sản thuê tài chính-225",
                        "GTCL tài sản cố định vô hình-226", "Nguyên giá TSCĐ vô hình-227", 
                        "Khấu khao lũy kế TSCĐ vô hình-228", "Xây dựng cơ bản dở dang (trước 2015)-2420"]
    lst_giatri_rong_taisan_dautu1 = ["Nguyên giá tài sản đầu tư-229", "Khấu hao lũy kế tài sản đầu tư-231"]
    lst_taisan_dodang_daihan1 = ["Chi phí sản xuất, kinh doanh dở dang dài hạn-241", "Xây dựng cơ bản dở dang-242"]
    lst_dautu_daihan1 = ["Đầu tư vào các công ty con-251", "Đầu tư vào các công ty liên kết-252",
                        "Đầu tư dài hạn khác-253", "Dự phòng giảm giá đầu tư dài hạn-254", 
                        "Lợi thế thương mại (trước 2015)-2690"]
    lst_taisan_daihan_khac1 = ["Trả trước dài hạn-261", "Thuế thu nhập hoãn lại phải thu-262", 
                            "Thiết bị, vật tư, phụ tùng thay thế dài hạn-263", "Các tài sản dài hạn khác-268",
                            "Lợi thế thương mại-269"]
    lst_nonganhan1 = ["Phải trả người bán-311", "Người mua trả tiền trước-312", "Thuế và các khoản phải trả Nhà nước-313",
                    "Phải trả người lao động-314", "Chi phí phải trả-315", "Phải trả nội bộ-316", "Phải trả về xây dựng cơ bản-317",
                    "Doanh thu chưa thực hiện ngắn hạn-318", "Phải trả khác-319", "Vay ngắn hạn-320", 
                    "Dự phòng các khoản phải trả ngắn hạn-321", "Quỹ khen thưởng, phúc lợi-322", "Quỹ bình ổn giá-323"]
    lst_nodaihan1 = ["Phải trả nhà cung cấp dài hạn-331", "Người mua trả tiền trước dài hạn-332", "Chi phí phải trả dài hạn-333",
                    "Phải trả nội bộ về vốn kinh doanh-334", "Phải trả nội bộ dài hạn-335", "Doanh thu chưa thực hiên-336",
                    "Phải trả dài hạn khác-337", "Vay dài hạn-338", "Trái phiếu chuyển đổi-339", "Cổ phiếu ưu đãi_ndh-340",
                    "Thuế thu nhập hoãn lại phải trả-341", "Dự phòng trợ cấp thôi việc-351", "Dự phòng các khoản công nợ dài hạn-342",
                    "Quỹ phát triển khoa học công nghệ-343"]
    lst_von_va_cacquy1 = ["Vốn góp-411", "Cổ phiếu phổ thông-412", "Cổ phiếu ưu đãi_vcsh-413", "Thặng dư vốn cổ phần-414",
                        "Quyền chọn chuyển đổi trái phiếu-415", "Vốn khác-416", "Cổ phiếu quỹ-417", "Chênh lệch đánh giá lại tài sản-418",
                        "Chênh lệch tỷ giá-419", "Quỹ đầu tư và phát triển-420", "Quỹ hỗ trợ sắp xếp doanh nghiệp-421",
                        "Quỹ dự phòng tài chính-422", "Quỹ khác-423", "Lãi chưa phân phối-424", "LNST chưa phân phối lũy kế đến cuối kỳ trước-425",
                        "LNST chưa phân phối kỳ này-426", "Lợi ích cổ đông không kiểm soát-427"]
    lst_von_nhanuoc_quykhac1 = ["Quỹ khen thưởng, phúc lợi (trước 2010)-430", "Vốn ngân sách nhà nước-431", "Nguồn kinh phí đã hình thành TSCĐ-432", 
                                "LỢI ÍCH CỦA CỔ ĐÔNG THIỂU SỐ-434"]
    
    lst_tien_tuongduong_tien = [i[:i.index("-")] for i in lst_tien_tuongduong_tien1]
    lst_giatrithuan_dautu_nganhan = [i[:i.index("-")] for i in lst_giatrithuan_dautu_nganhan1]
    lst_cackhoan_phaithu = [i[:i.index("-")] for i in lst_cackhoan_phaithu1]
    lst_hangtongkho_rong = [i[:i.index("-")] for i in lst_hangtongkho_rong1]
    lst_taisan_luudong_khac = [i[:i.index("-")] for i in lst_taisan_luudong_khac1]
    lst_phaithu_daihan = [i[:i.index("-")] for i in lst_phaithu_daihan1]
    lst_taisan_codinh = [i[:i.index("-")] for i in lst_taisan_codinh1]
    lst_giatri_rong_taisan_dautu = [i[:i.index("-")] for i in lst_giatri_rong_taisan_dautu1]
    lst_taisan_dodang_daihan = [i[:i.index("-")] for i in lst_taisan_dodang_daihan1]
    lst_dautu_daihan = [i[:i.index("-")] for i in lst_dautu_daihan1]
    lst_taisan_daihan_khac = [i[:i.index("-")] for i in lst_taisan_daihan_khac1]
    lst_nonganhan = [i[:i.index("-")] for i in lst_nonganhan1]
    lst_nodaihan = [i[:i.index("-")] for i in lst_nodaihan1]
    lst_von_va_cacquy = [i[:i.index("-")] for i in lst_von_va_cacquy1]
    lst_von_nhanuoc_quykhac = [i[:i.index("-")] for i in lst_von_nhanuoc_quykhac1]
    
    lst_tong_hop = lst_tien_tuongduong_tien1 + lst_giatrithuan_dautu_nganhan1 + lst_cackhoan_phaithu1 + lst_hangtongkho_rong1 + lst_taisan_luudong_khac1 \
                    + lst_phaithu_daihan1 + lst_taisan_codinh1 + lst_giatri_rong_taisan_dautu1 + lst_taisan_dodang_daihan1 + lst_dautu_daihan1 \
                    + lst_taisan_daihan_khac1 + lst_nonganhan1 + lst_nodaihan1 + lst_von_va_cacquy1 + lst_von_nhanuoc_quykhac1
    
    df_tong_hop_lv4 = pd.DataFrame(lst_tong_hop, columns=["Lv4"])
    df_tong_hop_lv4 = df_tong_hop_lv4["Lv4"].str.split('-', expand=True)
    df_tong_hop_lv4.columns = ["Item", "Lv4_ID"]

    
    def phan_loai_lv2(row):
        item = row.loc["Item"]

        if item in lst_tien_tuongduong_tien:
            return "I. Tiền và tương đương tiền"
        
        elif item in lst_giatrithuan_dautu_nganhan:
            return "II. Giá trị thuần đầu tư ngắn hạn"
        
        elif item in lst_cackhoan_phaithu:
            return "III. Các khoản phải thu"
        
        elif item in lst_hangtongkho_rong:
            return "IV. Hàng tồn kho, ròng"
        
        elif item in lst_taisan_luudong_khac:
            return "V. Tài sản lưu động khác"
        
        elif item in lst_phaithu_daihan:
            return "I. Phải thu dài hạn"
        
        elif item in lst_taisan_codinh:
            return "II. Tài sản cố định"
        
        elif item in lst_giatri_rong_taisan_dautu:
            return "III. Giá trị ròng tài sản đầu tư"
        
        elif item in lst_taisan_dodang_daihan:
            return "IV. Tài sản dở dang dài hạn"
        
        elif item in lst_dautu_daihan:
            return "V. Đầu tư dài hạn"
        
        elif item in lst_taisan_daihan_khac:
            return "VI. Tài sản dài hạn khác"
        
        elif item in lst_nonganhan:
            return "I. Nợ ngắn hạn"
        
        elif item in lst_nodaihan:
            return "II. Nợ dài hạn"
        
        elif item in lst_von_va_cacquy:
            return "I. Vốn và các quỹ"
        
        elif item in lst_von_nhanuoc_quykhac:
            return "II. Vốn Ngân sách nhà nước và quỹ khác"

        else: return

    balance_sheet["Lv3"] = balance_sheet.apply(phan_loai_lv2, axis="columns")
    balance_sheet.columns = ["Lv4", "Lv2", "Lv1", "Lv3"]
    balance_sheet = balance_sheet[["Lv1", "Lv2", "Lv3", "Lv4"]]
    balance_sheet = balance_sheet.iloc[3:]
    balance_sheet["Lv1_ID"] = balance_sheet.apply(lambda row:"270" if row['Lv1'] == "TỔNG CỘNG TÀI SẢN (270 = 100 + 200)" else "440", axis=1)

    def phan_loai_lv3(row):
        item = row.loc["Lv2"]

        if item == "A - TÀI SẢN NGẮN HẠN":
            return "100"
        
        elif item == "B - TÀI SẢN DÀI HẠN":
            return "200"
        
        elif item == "C - NỢ PHẢI TRẢ":
            return "300"
        
        elif item == "D - VỐN CHỦ SỞ HỮU":
            return "400"

        else: return

    balance_sheet["Lv2_ID"] = balance_sheet.apply(phan_loai_lv3, axis="columns")
    balance_sheet = balance_sheet.merge(df_tong_hop_lv4, how="left", left_on="Lv4", right_on="Item")
    balance_sheet = balance_sheet.drop("Item", axis="columns")
    balance_sheet = balance_sheet.dropna(how="any")

    def phan_loai_lv4(row):
        item = row.loc["Lv3"]

        if item == "I. Tiền và tương đương tiền":
            return "110"
        
        elif item == "II. Giá trị thuần đầu tư ngắn hạn":
            return "120"
        
        elif item == "III. Các khoản phải thu":
            return "130"
        
        elif item == "IV. Hàng tồn kho, ròng":
            return "140"
        
        elif item == "V. Tài sản lưu động khác":
            return "150"
        
        elif item == "I. Phải thu dài hạn":
            return "210"
        
        elif item == "II. Tài sản cố định":
            return "220"
        
        elif item == "III. Giá trị ròng tài sản đầu tư":
            return "230"
        
        elif item == "IV. Tài sản dở dang dài hạn":
            return "240"
        
        elif item == "V. Đầu tư dài hạn":
            return "250"
        
        elif item == "VI. Tài sản dài hạn khác":
            return "260"
        
        elif item == "I. Nợ ngắn hạn":
            return "310"
        
        elif item == "II. Nợ dài hạn":
            return "330"
        
        elif item == "I. Vốn và các quỹ":
            return "410"
        
        elif item == "II. Vốn Ngân sách nhà nước và quỹ khác":
            return "499"

        else: return

    balance_sheet["Lv3_ID"] = balance_sheet.apply(phan_loai_lv4, axis="columns")
    balance_sheet = balance_sheet[["Lv1_ID", "Lv1", "Lv2_ID", "Lv2", "Lv3_ID", "Lv3", "Lv4_ID", "Lv4"]]
    balance_sheet["Lv1_S"] = balance_sheet.apply(lambda row:"Tài sản" if row['Lv1_ID'] == "270" else "Nguồn vốn", axis=1)

    def phan_loai_lv5(row):
        item = row.loc["Lv2_ID"]

        if item == "100":
            return "Tài sản ngắn hạn"
        
        elif item == "200":
            return "Tài sản dài hạn"
        
        elif item == "300":
            return "Nợ phải trả"
        
        elif item == "400":
            return "Vốn CSH"
        
        else: return

    balance_sheet["Kết_cấu_CĐKT"] = balance_sheet.apply(phan_loai_lv5, axis="columns")

    def phan_loai_lv6(row):
        item = row.loc["Lv3"]

        if item in ["I. Tiền và tương đương tiền", "II. Giá trị thuần đầu tư ngắn hạn",
                    "III. Các khoản phải thu", "IV. Hàng tồn kho, ròng", "V. Tài sản lưu động khác"]:
            return "Tài sản ngắn hạn"
        
        elif item in ["I. Phải thu dài hạn", "II. Tài sản cố định", "III. Giá trị ròng tài sản đầu tư",
                    "IV. Tài sản dở dang dài hạn", "V. Đầu tư dài hạn", "VI. Tài sản dài hạn khác"]:
            return "Tài sản dài hạn"
        
        elif item == "I. Nợ ngắn hạn":
            return "Vốn ngắn hạn"
        
        elif item in ["II. Nợ dài hạn", "I. Vốn và các quỹ", "II. Vốn Ngân sách nhà nước và quỹ khác"]:
            return "Vốn dài hạn"
        
        else: return

    balance_sheet["Kỳ_TS_NV"] = balance_sheet.apply(phan_loai_lv6, axis="columns")
    lst_balance_sheet.append(balance_sheet)

    balance_sheet.to_excel(f"D:\\Tin hoc\\Project\\Báo cáo tài chính Power BI\\BCTC\\dim_balance_sheet\\dim_balance_sheet_draft_{stock_id}.xlsx", index=False)


In [ ]:
fact_balance_sheet = []
for i in files:
    stock = i[:3]
    dim_balance_sheet(stock)
    fact_balance_sheet.append(df_new)

fact_balance_sheet = list(zip([i[:3] for i in files], fact_balance_sheet))
df_dim_balance_sheet = lst_balance_sheet[-1]

fact_balance_sheet_all = []
for m,n in enumerate(fact_balance_sheet):
    fact_name = n[0]
    df_fact = n[1]
    df_fact["Value"] = df_fact["Value"].fillna(value="null")
    df_fact["phan_loai"] = df_fact["Value"].apply(type)
    df_fact = df_fact[df_fact["phan_loai"]!=str]
    df_fact = df_fact[["Item", "Year", "Value"]]
    df_fact["stock"] = fact_name
    df_fact["Item"] = df_fact["Item"].str.strip()
    df_fact.to_excel(f'D:\\Tin hoc\\Project\\Báo cáo tài chính Power BI\\BCTC\\fact_balance_sheet\\fact_{fact_name}.xlsx', index=False)
    fact_balance_sheet_all.append(df_fact)

fact_all = pd.concat(fact_balance_sheet_all)
lst_lv4 = list(lst_balance_sheet[-1]["Lv4"].unique())
fact_all = fact_all[fact_all["Item"].isin(lst_lv4)]
fact_all = fact_all.reset_index(drop=True)
fact_all = fact_all.merge(df_dim_balance_sheet[["Lv4_ID", "Lv4"]], how="left", left_on="Item", right_on="Lv4")
fact_all = fact_all.drop("Lv4", axis="columns")

lst_thut_dong = ["Nguyên giá TSCĐ hữu hình", "Khấu hao lũy kế TSCĐ hữu hình", "Nguyên giá tài sản thuê tài chính", "Khấu hao lũy kế tài sản thuê tài chính",
                 "Nguyên giá TSCĐ vô hình", "Khấu khao lũy kế TSCĐ vô hình", "Cổ phiếu phổ thông", "Cổ phiếu ưu đãi_vcsh", 
                 "LNST chưa phân phối lũy kế đến cuối kỳ trước", "LNST chưa phân phối kỳ này"]

df_dim_balance_sheet["Lv4"] = df_dim_balance_sheet.apply(lambda row:"-" * 3 + row['Lv4'] if row['Lv4'] in lst_thut_dong else row['Lv4'], axis=1)

In [5]:
list_bkqkd = []
skiprows = [i for i in range(1,10)]
for index, file in enumerate(files):
    df_full_in = pd.read_excel(os.path.join(path, file), skiprows=lambda x: x in skiprows, sheet_name = None)
    bkqkd = list(df_full_in.keys())[1]
    df_full_in = df_full_in[bkqkd]
    columns = list(df_full_in.iloc[0].unique())
    df_full_in.columns = ["Item"] + columns[1:]
    df_full_in["Stock"] = file[:3]
    df_full_in = df_full_in.iloc[4:29]
    list_bkqkd.append(df_full_in)

In [6]:
dim_income_statement = list_bkqkd[0]["Item"]
dim_income_statement = dim_income_statement.to_frame().reset_index(drop=True)
dim_income_statement.columns = ["Item_dim"]
dim_income_statement["Id"] = pd.DataFrame(list(dim_income_statement.index))
dim_income_statement["Item_dim"] = dim_income_statement["Item_dim"].str.strip()

In [7]:
lst_fact_kqkd = []
for i in list_bkqkd:
    i = i.melt(id_vars=["Item", "Stock"],var_name="Year", value_name="Value")
    lst_fact_kqkd.append(i)
df_fact_kqkd = pd.DataFrame(pd.concat(lst_fact_kqkd))
df_fact_kqkd = df_fact_kqkd.reset_index(drop=True)
df_fact_kqkd["Item"] = df_fact_kqkd["Item"].str.strip()
df_fact_kqkd = df_fact_kqkd.merge(dim_income_statement, how="left", left_on="Item", right_on="Item_dim")
df_fact_kqkd = df_fact_kqkd.drop("Item_dim", axis="columns")

lst_thut_dong_income = ["Trong đó: Chi phí lãi vay", "Thu nhập khác", "Chi phí khác", "Thuế thu nhập doanh nghiệp – hiện thời", "Thuế thu nhập doanh nghiệp – hoãn lại"]
dim_income_statement["Item_dim"] = dim_income_statement.apply(lambda row:"-" * 3 + row['Item_dim'] if row['Item_dim'] in lst_thut_dong_income else row['Item_dim'], axis=1)

In [ ]:
list_lctt = []
skiprows = [i for i in range(1,10)]
for index, file in enumerate(files):
    df_full_cf = pd.read_excel(os.path.join(path, file), skiprows=lambda x: x in skiprows, sheet_name = None)
    lctt = list(df_full_cf.keys())[2]
    df_full_cf = df_full_cf[lctt]
    columns = list(df_full_cf.iloc[0].unique())
    df_full_cf.columns = ["Item"] + columns[1:]
    df_full_cf["Stock"] = file[:3]
    df_full_cf = df_full_cf.iloc[4:45]
    list_lctt.append(df_full_cf)

dim_cash_flow = list_lctt[0]["Item"]
dim_cash_flow = dim_cash_flow.to_frame().reset_index(drop=True)
dim_cash_flow.columns = ["Item_dim"]
dim_cash_flow["Lv2_ID"] = pd.DataFrame(list(dim_cash_flow.index))
dim_cash_flow["Item_dim"] = dim_cash_flow["Item_dim"].str.strip()

lst_hd_sxkd = ["Lợi nhuận/(lỗ) trước thuế", "Khấu hao TSCĐ và BĐSĐT", "Phân bổ lợi thế thương mại", "Chi phí dự phòng", "Lãi/lỗ chênh lệch tỷ giá hối đoái do đánh giá lại các khoản mục tiền tệ có gốc ngoại tệ",
               "Lãi/(lỗ) từ thanh lý tài sản cố định", "(Lãi)/lỗ từ hoạt động đầu tư", "Chi phí lãi vay", "Thu lãi và cổ tức", "Các khoản điều chỉnh khác",
               "Lợi nhuận/(lỗ) từ hoạt động kinh doanh trước những thay đổi vốn lưu động", "(Tăng)/giảm các khoản phải thu", "(Tăng)/giảm hàng tồn kho",
               "Tăng/(giảm) các khoản phải trả", "(Tăng)/giảm chi phí trả trước", "(Tăng)/giảm chứng khoán kinh doanh", "Tiền lãi vay đã trả", "Thuế thu nhập doanh nghiệp đã nộp",
               "Tiền thu khác từ hoạt động kinh doanh", "Tiền chi khác cho hoạt động kinh doanh", 
               "Lưu chuyển tiền tệ ròng từ các hoạt động sản xuất kinh doanh"]

lst_hd_hddt = ["Lưu chuyển tiền thuần từ hoạt động đầu tư", "Tiền chi để mua sắm, xây dựng TSCĐ và các tài sản dài hạn khác",
               "Tiền thu từ thanh lý, nhượng bán TSCĐ và các tài sản dài hạn khác", "Tiền chi cho vay, mua các công cụ nợ của đơn vị khác",
               "Tiền thu hồi cho vay, bán lại các công cụ nợ của đơn vị khác", "Tiền chi đầu tư góp vốn vào đơn vị khác",
               "Tiền thu hồi đầu tư góp vốn vào đơn vị khác", "Tiền thu lãi cho vay, cổ tức và lợi nhuận được chia"]

lst_hd_hdtc = ["Lưu chuyển tiền thuần từ hoạt động tài chính", "Tiền thu từ phát hành cổ phiếu, nhận vốn góp của chủ sở hữu", 
               "Tiền chi trả vốn góp cho các chủ sở hữu, mua lại cổ phiếu của doanh nghiệp đã phát hành", "Tiền thu được các khoản đi vay",
               "Tiền trả nợ gốc vay", "Tiền trả nợ gốc thuê tài chính", "Cổ tức, lợi nhuận đã trả cho chủ sở hữu", "Tiền lãi đã nhận"]

def phan_loai_cf_1(row):
    item = row.loc["Item_dim"]

    if item in lst_hd_sxkd:
        return "I. Lưu chuyển tiền tệ ròng từ các hoạt động sản xuất kinh doanh"
        
    elif item in lst_hd_hddt:
        return "II. Lưu chuyển tiền tệ ròng từ hoạt động đầu tư"
        
    elif item in lst_hd_hdtc:
        return "III. Lưu chuyển tiền tệ từ hoạt động tài chính"
        
    elif item == "Lưu chuyển tiền thuần trong kỳ":
        return "Lưu chuyển tiền thuần trong kỳ"
        
    elif item == "Tiền và tương đương tiền đầu kỳ":
        return "Tiền và tương đương tiền đầu kỳ"
        
    elif item == "Ảnh hưởng của thay đổi tỷ giá hối đoái quy đổi ngoại tệ":
        return "Ảnh hưởng của thay đổi tỷ giá hối đoái quy đổi ngoại tệ"
    
    elif item == "Tiền và tương đương tiền cuối kỳ":
        return "Tiền và tương đương tiền cuối kỳ"

    else: return

dim_cash_flow["Lv1"] = dim_cash_flow.apply(phan_loai_cf_1, axis="columns")

def phan_loai_cf_2(row):
    item = row.loc["Lv1"]

    if item == "I. Lưu chuyển tiền tệ ròng từ các hoạt động sản xuất kinh doanh":
        return "20"
    
    elif item == "II. Lưu chuyển tiền tệ ròng từ hoạt động đầu tư":
        return "30"

    elif item == "III. Lưu chuyển tiền tệ từ hoạt động tài chính":
        return "40"
    
    elif item == "Lưu chuyển tiền thuần trong kỳ":
        return "50"
    
    elif item == "Tiền và tương đương tiền đầu kỳ":
        return "60"
    
    elif item == "Ảnh hưởng của thay đổi tỷ giá hối đoái quy đổi ngoại tệ":
        return "61"
    
    elif item == "Tiền và tương đương tiền cuối kỳ":
        return "70"
    
    else: return

dim_cash_flow["Lv1_ID"] = dim_cash_flow.apply(phan_loai_cf_2, axis="columns")
dim_cash_flow = dim_cash_flow[["Lv1_ID", "Lv1", "Lv2_ID", "Item_dim"]]
dim_cash_flow.columns = ["Lv1_ID", "Lv1", "Lv2_ID", "Lv2"]

In [9]:
lst_fact_lctt = []
for i in list_lctt:
    i = i.melt(id_vars=["Item", "Stock"],var_name="Year", value_name="Value")
    lst_fact_lctt.append(i)
df_fact_lctt = pd.DataFrame(pd.concat(lst_fact_lctt))
df_fact_lctt = df_fact_lctt.reset_index(drop=True)
df_fact_lctt["Item"] = df_fact_lctt["Item"].str.strip()
df_fact_lctt = df_fact_lctt.merge(dim_cash_flow, how="left", left_on="Item", right_on="Lv2")
df_fact_lctt = df_fact_lctt.drop(["Lv2", "Lv1", "Lv1_ID"], axis="columns")

In [10]:
lst_thut_dong_cf = ["(Tăng)/giảm các khoản phải thu", "(Tăng)/giảm hàng tồn kho", "Tăng/(giảm) các khoản phải trả", "(Tăng)/giảm chi phí trả trước",
                        "(Tăng)/giảm chứng khoán kinh doanh", "Tiền lãi vay đã trả", "Thuế thu nhập doanh nghiệp đã nộp", "Tiền thu khác từ hoạt động kinh doanh",
                        "Tiền chi khác cho hoạt động kinh doanh", "Tiền chi để mua sắm, xây dựng TSCĐ và các tài sản dài hạn khác",
                        "Tiền thu từ thanh lý, nhượng bán TSCĐ và các tài sản dài hạn khác", "Tiền chi cho vay, mua các công cụ nợ của đơn vị khác",
                        "Tiền thu hồi cho vay, bán lại các công cụ nợ của đơn vị khác", "Tiền chi đầu tư góp vốn vào đơn vị khác",
                        "Tiền thu hồi đầu tư góp vốn vào đơn vị khác", "Tiền thu lãi cho vay, cổ tức và lợi nhuận được chia",
                        "Tiền thu từ phát hành cổ phiếu, nhận vốn góp của chủ sở hữu", "Tiền chi trả vốn góp cho các chủ sở hữu, mua lại cổ phiếu của doanh nghiệp đã phát hành",
                        "Tiền thu được các khoản đi vay", "Tiền trả nợ gốc vay", "Tiền trả nợ gốc thuê tài chính", "Cổ tức, lợi nhuận đã trả cho chủ sở hữu", "Tiền lãi đã nhận"]
dim_cash_flow["Lv2"] = dim_cash_flow.apply(lambda row:"-" * 3 + row['Lv2'] if row['Lv2'] in lst_thut_dong_cf else row['Lv2'], axis=1)

In [11]:
df_stock = pd.DataFrame(fact_all["stock"].unique())
df_stock.columns = ["Stock"]
df_stock["Phan_loai_1"] = 'Phi tài chính'
df_stock["Phan_loai_2"] = 'Sản xuất hàng hoá'
df_stock

,Stock,Phan_loai_1,Phan_loai_2
0,BFC,Phi tài chính,Sản xuất hàng hoá
1,BSR,Phi tài chính,Sản xuất hàng hoá
2,DCM,Phi tài chính,Sản xuất hàng hoá
3,DPM,Phi tài chính,Sản xuất hàng hoá
4,DPR,Phi tài chính,Sản xuất hàng hoá
5,GMD,Phi tài chính,Sản xuất hàng hoá
6,HAH,Phi tài chính,Sản xuất hàng hoá
7,HPG,Phi tài chính,Sản xuất hàng hoá
8,IMP,Phi tài chính,Sản xuất hàng hoá
9,NKG,Phi tài chính,Sản xuất hàng hoá


In [12]:
dim_lcct = {
    "1": "Dư tiền đầu kỳ",
    "2": "Dòng tiền kinh doanh",
    "3": "Dòng tiền đầu tư",
    "4": "Dòng tiền tài chính",
    "5": "Chênh lệch tỷ giá",
    "6": "Dư tiền cuối kỳ"
}
dim_lcct = pd.DataFrame.from_dict(dim_lcct, orient='index')
dim_lcct.columns = ["Item"]
dim_lcct["Id"] = list(dim_lcct.index)
dim_lcct

,Item,Id
1,Dư tiền đầu kỳ,1
2,Dòng tiền kinh doanh,2
3,Dòng tiền đầu tư,3
4,Dòng tiền tài chính,4
5,Chênh lệch tỷ giá,5
6,Dư tiền cuối kỳ,6


In [13]:
lst_year = [int(i) for i in fact_all["Year"].unique()]
first_year = sorted(lst_year)[0]
last_year = sorted(lst_year)[-1]
first_year = str(first_year) + "-01-01"
last_year = str(last_year) + "-12-31"
dim_view_dk = pd.date_range(start=first_year, end=last_year, freq="MS")
dim_view_dk = dim_view_dk.to_frame()
dim_view_dk = dim_view_dk.reset_index(drop=True)
dim_view_dk.columns = ["Date"]
dim_view_dk["Dau_ky"] = dim_view_dk["Date"] + pd.DateOffset(months=1)
dim_view_dk["Cuoi_ky"] = dim_view_dk["Date"] + pd.tseries.offsets.MonthEnd()
dim_view_dk

,Date,Dau_ky,Cuoi_ky
0,2015-01-01,2015-02-01,2015-01-31
1,2015-02-01,2015-03-01,2015-02-28
2,2015-03-01,2015-04-01,2015-03-31
3,2015-04-01,2015-05-01,2015-04-30
4,2015-05-01,2015-06-01,2015-05-31
...,...,...,...
115,2024-08-01,2024-09-01,2024-08-31
116,2024-09-01,2024-10-01,2024-09-30
117,2024-10-01,2024-11-01,2024-10-31
118,2024-11-01,2024-12-01,2024-11-30


In [14]:
lst_hqhd = [("1", "Kết quả hoạt động", "1", "LN sau thuế"), ("1", "Kết quả hoạt động", "2", "EBT"), ("1", "Kết quả hoạt động", "3", "EBIT"),
            ("1", "Kết quả hoạt động", "4", "LN gộp"), ("1", "Kết quả hoạt động", "5", "Doanh thu thuần"), ("2", "Nguồn", "6", "Tổng Tài sản BQ"),
            ("2", "Nguồn", "7", "Vốn CSH BQ")]
dim_hqhd = pd.DataFrame(lst_hqhd)
dim_hqhd.columns = ["ID_1", "KM_1", "ID_2", "KM_2"]
dim_hqhd

,ID_1,KM_1,ID_2,KM_2
0,1,Kết quả hoạt động,1,LN sau thuế
1,1,Kết quả hoạt động,2,EBT
2,1,Kết quả hoạt động,3,EBIT
3,1,Kết quả hoạt động,4,LN gộp
4,1,Kết quả hoạt động,5,Doanh thu thuần
5,2,Nguồn,6,Tổng Tài sản BQ
6,2,Nguồn,7,Vốn CSH BQ


In [15]:
lst_kctsnv = [("1", "Tài sản", "1", "Tài sản ngắn hạn"), ("1", "Tài sản", "2", "Tài sản dài hạn"), ("2", "Nguồn vốn", "3", "Vốn ngắn hạn"),
            ("2", "Nguồn vốn", "4", "Vốn dài hạn"), ("3", "Vốn lưu động", "5", "Vốn lưu động")]
dim_kctsnv= pd.DataFrame(lst_kctsnv)
dim_kctsnv.columns = ["ID_1", "KM_1", "ID_2", "KM_2"]
dim_kctsnv

,ID_1,KM_1,ID_2,KM_2
0,1,Tài sản,1,Tài sản ngắn hạn
1,1,Tài sản,2,Tài sản dài hạn
2,2,Nguồn vốn,3,Vốn ngắn hạn
3,2,Nguồn vốn,4,Vốn dài hạn
4,3,Vốn lưu động,5,Vốn lưu động


In [16]:
dim_view_nam = pd.DataFrame(pd.date_range(start=first_year, end=last_year, freq="YS"), columns=["Date"])
dim_view_nam["Dau_ky"] = pd.date_range(start=first_year, end=last_year, freq="YS")

lst_date_dau_ky = dim_view_nam["Dau_ky"].unique()
lst_date_dau_ky_new = []
for i in lst_date_dau_ky:
    k = dt.date((i.year + 1), (i.month), (i.day))
    lst_date_dau_ky_new.append(k)
df_date_dau_ky_new = pd.DataFrame(lst_date_dau_ky_new)

dim_view_nam["Dau_ky"] = df_date_dau_ky_new
dim_view_nam["Cuoi_ky"] = pd.date_range(start=first_year, end=last_year, freq="Y")
dim_view_nam

,Date,Dau_ky,Cuoi_ky
0,2015-01-01,2016-01-01,2015-12-31
1,2016-01-01,2017-01-01,2016-12-31
2,2017-01-01,2018-01-01,2017-12-31
3,2018-01-01,2019-01-01,2018-12-31
4,2019-01-01,2020-01-01,2019-12-31
5,2020-01-01,2021-01-01,2020-12-31
6,2021-01-01,2022-01-01,2021-12-31
7,2022-01-01,2023-01-01,2022-12-31
8,2023-01-01,2024-01-01,2023-12-31
9,2024-01-01,2025-01-01,2024-12-31


In [18]:
df_dim_balance_sheet["ETL_date"] = str(dt.datetime.now())
df_dim_balance_sheet['danh_dau'] = df_dim_balance_sheet.apply(lambda row:"0" if row['Lv4'].startswith('---') else "1", axis=1)
df_dim_balance_sheet = df_dim_balance_sheet[["Lv1_ID", "Lv1", "Lv2_ID", "Lv2", "Lv3_ID", "Lv3", "Lv4_ID", "Lv4", "Lv1_S", "Kết_cấu_CĐKT", "Kỳ_TS_NV", "danh_dau", "ETL_date"]]

query = '''if (OBJECT_ID('dim_balance_sheet') is not null)
begin
	print('abcd')
end
else
begin
	create table dim_balance_sheet
	(
		[Lv1_ID] nvarchar(255),
		[Lv1] nvarchar(255),
        [Lv2_ID] nvarchar(255),
        [Lv2] nvarchar(255),
        [Lv3_ID] nvarchar(255),
        [Lv3] nvarchar(255),
        [Lv4_ID] nvarchar(255),
        [Lv4] nvarchar(255),
        [Lv1_S] nvarchar(255),
        [Kết_cấu_CĐKT] nvarchar(255),
        [Kỳ_TS_NV] nvarchar(255),
        [danh_dau] varchar(5),
        [ETL_date] varchar(50)
	);
end
'''
cursor.execute(query)
conn.commit()

In [19]:
df_dim_balance_sheet.to_csv(f"D:\\Tin hoc\\Project\\Báo cáo tài chính Power BI\\BCTC\\db_file_financial_statement_analysis\\dim_balance_sheet.txt", index=False)
with open(r'D:\Tin hoc\Project\Báo cáo tài chính Power BI\BCTC\db_file_financial_statement_analysis\dim_balance_sheet.txt', newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE dim_balance_sheet')
    cursor.executemany(''' INSERT INTO dim_balance_sheet Values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', reader)
    conn.commit()

In [20]:
fact_all["ETL_date"] = str(dt.datetime.now())
fact_all["Year"] = fact_all["Year"].astype("datetime64[ns]")

query = '''if (OBJECT_ID('fact_balance_sheet') is not null)
begin
	print('abcd')
end
else
begin
	create table fact_balance_sheet
	(
		[Item] nvarchar(255),
		[Year] date,
        [Value] float,
        [Stock] varchar(5),
        [Lv4_ID] varchar(10),
        [ETL_date] varchar(50)
	);
end
'''
cursor.execute(query)
conn.commit()

In [21]:
fact_all.to_csv(f"D:\\Tin hoc\\Project\\Báo cáo tài chính Power BI\\BCTC\\db_file_financial_statement_analysis\\fact_all.txt", index=False)
with open(r'D:\Tin hoc\Project\Báo cáo tài chính Power BI\BCTC\db_file_financial_statement_analysis\fact_all.txt', newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE fact_balance_sheet')
    cursor.executemany(''' INSERT INTO fact_balance_sheet Values (?, ?, ?, ?, ?, ?)''', reader)
    conn.commit()

In [22]:
dim_income_statement["ETL_date"] = str(dt.datetime.now())

query = '''if (OBJECT_ID('dim_income_statement') is not null)
begin
	print('abcd')
end
else
begin
	create table dim_income_statement
	(
		[Item_dim] nvarchar(255),
		[Id] varchar(10),
        [ETL_date] varchar(50)
	);
end
'''
cursor.execute(query)
conn.commit()

In [23]:
dim_income_statement.to_csv(f"D:\\Tin hoc\\Project\\Báo cáo tài chính Power BI\\BCTC\\db_file_financial_statement_analysis\\dim_income_statement.txt", index=False)
with open(r'D:\Tin hoc\Project\Báo cáo tài chính Power BI\BCTC\db_file_financial_statement_analysis\dim_income_statement.txt', newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE dim_income_statement')
    cursor.executemany(''' INSERT INTO dim_income_statement Values (?, ?, ?)''', reader)
    conn.commit()

In [24]:
df_fact_kqkd["ETL_date"] = str(dt.datetime.now())
df_fact_kqkd["Year"] = df_fact_kqkd["Year"].astype("datetime64[ns]")

query = '''if (OBJECT_ID('fact_income_statement') is not null)
begin
	print('abcd')
end
else
begin
	create table fact_income_statement
	(
		[Item] nvarchar(255),
        [Stock] varchar(5),
		[Year] date,
        [Value] float,
        [ID] varchar(10),
        [ETL_date] varchar(50)
	);
end
'''
cursor.execute(query)
conn.commit()

In [25]:
df_fact_kqkd.to_csv(f"D:\\Tin hoc\\Project\\Báo cáo tài chính Power BI\\BCTC\\db_file_financial_statement_analysis\\fact_income_statement.txt", index=False)
with open(r'D:\Tin hoc\Project\Báo cáo tài chính Power BI\BCTC\db_file_financial_statement_analysis\fact_income_statement.txt', newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE fact_income_statement')
    cursor.executemany(''' INSERT INTO fact_income_statement Values (?, ?, ?, ?, ?, ?)''', reader)
    conn.commit()

In [26]:
dim_cash_flow["ETL_date"] = str(dt.datetime.now())

query = '''if (OBJECT_ID('dim_cash_flow') is not null)
begin
	print('abcd')
end
else
begin
	create table dim_cash_flow
	(
		[Lv1_ID] varchar(20),
        [Lv1] nvarchar(255),
        [Lv2_ID] varchar(20),
        [Lv2] nvarchar(255),
        [ETL_date] varchar(50)
	);
end
'''
cursor.execute(query)
conn.commit()

In [27]:
dim_cash_flow.to_csv(f"D:\\Tin hoc\\Project\\Báo cáo tài chính Power BI\\BCTC\\db_file_financial_statement_analysis\\dim_cash_flow.txt", index=False)
with open(r'D:\Tin hoc\Project\Báo cáo tài chính Power BI\BCTC\db_file_financial_statement_analysis\dim_cash_flow.txt', newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE dim_cash_flow')
    cursor.executemany(''' INSERT INTO dim_cash_flow Values (?, ?, ?, ?, ?)''', reader)
    conn.commit()

In [28]:
mask1 = df_fact_lctt["Lv2_ID"] == 20
mask2 = df_fact_lctt["Lv2_ID"] == 21
mask3 = df_fact_lctt["Lv2_ID"] == 29
df_fact_lctt = df_fact_lctt[~mask1 & ~mask2 & ~mask3] 
df_fact_lctt["ETL_date"] = str(dt.datetime.now())
df_fact_lctt["Year"] = df_fact_lctt["Year"].astype("datetime64[ns]")

query = '''if (OBJECT_ID('fact_cash_flow') is not null)
begin
	print('abcd')
end
else
begin
	create table fact_cash_flow
	(
		[Item] nvarchar(255),
        [Stock] varchar(5),
		[Year] date,
        [Value] float,
        [Lv2_ID] varchar(20),
        [ETL_date] varchar(50)
	);
end
'''
cursor.execute(query)
conn.commit()

In [29]:
df_fact_lctt[df_fact_lctt['Stock'] == 'DBD']

,Item,Stock,Year,Value,Lv2_ID,ETL_date


In [30]:
df_fact_lctt.to_csv(f"D:\\Tin hoc\\Project\\Báo cáo tài chính Power BI\\BCTC\\db_file_financial_statement_analysis\\fact_cash_flow.txt", index=False)
with open(r'D:\Tin hoc\Project\Báo cáo tài chính Power BI\BCTC\db_file_financial_statement_analysis\fact_cash_flow.txt', newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE fact_cash_flow')
    cursor.executemany(''' INSERT INTO fact_cash_flow Values (?, ?, ?, ?, ?, ?)''', reader)
    conn.commit()

In [31]:
df_image_url = pd.read_excel(r"D:\Tin hoc\Project\Báo cáo tài chính Power BI\BCTC\Financial Sample.xlsx", sheet_name="Managers")
df_stock = df_stock.merge(right=df_image_url, how="left", left_on="Stock", right_on="Stock")
df_stock["ETL_date"] = str(dt.datetime.now())

query = '''if (OBJECT_ID('dim_stock') is not null)
begin
	print('abcd')
end
else
begin
	create table dim_stock
	(
        [Stock] varchar(5),
		[Phan_loai_1] nvarchar(50),
        [Phan_loai_2] nvarchar(50),
        [Url_image] varchar(200),
        [ETL_date] varchar(50)
	);
end
'''
cursor.execute(query)
conn.commit()

In [32]:
df_stock.to_csv(f"D:\\Tin hoc\\Project\\Báo cáo tài chính Power BI\\BCTC\\db_file_financial_statement_analysis\\dim_stock.txt", index=False)
with open(r'D:\Tin hoc\Project\Báo cáo tài chính Power BI\BCTC\db_file_financial_statement_analysis\dim_stock.txt', newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE dim_stock')
    cursor.executemany(''' INSERT INTO dim_stock Values (?, ?, ?, ?, ?)''', reader)
    conn.commit()

In [33]:
dim_lcct

,Item,Id
1,Dư tiền đầu kỳ,1
2,Dòng tiền kinh doanh,2
3,Dòng tiền đầu tư,3
4,Dòng tiền tài chính,4
5,Chênh lệch tỷ giá,5
6,Dư tiền cuối kỳ,6


In [34]:
dim_lcct["ETL_date"] = str(dt.datetime.now())

query = '''if (OBJECT_ID('dim_lcct') is not null)
begin
	print('abcd')
end
else
begin
	create table dim_lcct
	(
        [Item] nvarchar(255),
		[Id] varchar(20),
        [ETL_date] varchar(50)
	);
end
'''
cursor.execute(query)
conn.commit()

In [35]:
dim_lcct.to_csv(f"D:\\Tin hoc\\Project\\Báo cáo tài chính Power BI\\BCTC\\db_file_financial_statement_analysis\\dim_lcct.txt", index=False)
with open(r'D:\Tin hoc\Project\Báo cáo tài chính Power BI\BCTC\db_file_financial_statement_analysis\dim_lcct.txt', newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE dim_lcct')
    cursor.executemany(''' INSERT INTO dim_lcct Values (?, ?, ?)''', reader)
    conn.commit()

In [36]:
dim_view_dk["ETL_date"] = str(dt.datetime.now())

query = '''if (OBJECT_ID('dim_view_dk') is not null)
begin
	print('abcd')
end
else
begin
	create table dim_view_dk
	(
        [Date] datetime2,
		[Dau_ky] datetime2,
        [Cuoi_ky] datetime2,
        [ETL_date] varchar(50)
	);
end
'''
cursor.execute(query)
conn.commit()

In [37]:
dim_view_dk.to_csv(f"D:\\Tin hoc\\Project\\Báo cáo tài chính Power BI\\BCTC\\db_file_financial_statement_analysis\\dim_view_dk.txt", index=False)
with open(r'D:\Tin hoc\Project\Báo cáo tài chính Power BI\BCTC\db_file_financial_statement_analysis\dim_view_dk.txt', newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE dim_view_dk')
    cursor.executemany(''' INSERT INTO dim_view_dk Values (?, ?, ?, ?)''', reader)
    conn.commit()

In [38]:
dim_view_nam["ETL_date"] = str(dt.datetime.now())

query = '''if (OBJECT_ID('dim_view_nam') is not null)
begin
	print('abcd')
end
else
begin
	create table dim_view_nam
	(
        [Date] datetime2,
		[Dau_ky] datetime2,
        [Cuoi_ky] datetime2,
        [ETL_date] varchar(50)
	);
end
'''
cursor.execute(query)
conn.commit()

In [39]:
dim_view_nam.to_csv(f"D:\\Tin hoc\\Project\\Báo cáo tài chính Power BI\\BCTC\\db_file_financial_statement_analysis\\dim_view_nam.txt", index=False)
with open(r'D:\Tin hoc\Project\Báo cáo tài chính Power BI\BCTC\db_file_financial_statement_analysis\dim_view_nam.txt', newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE dim_view_nam')
    cursor.executemany(''' INSERT INTO dim_view_nam Values (?, ?, ?, ?)''', reader)
    conn.commit()

In [40]:
dim_hqhd["ETL_date"] = str(dt.datetime.now())

query = '''if (OBJECT_ID('dim_hqhd') is not null)
begin
	print('abcd')
end
else
begin
	create table dim_hqhd
	(
        [ID_1] varchar(10),
		[KM_1] nvarchar(100),
        [ID_2] varchar(10),
        [KM_2] nvarchar(100),
        [ETL_date] varchar(50)
	);
end
'''
cursor.execute(query)
conn.commit()

In [41]:
dim_hqhd.to_csv(f"D:\\Tin hoc\\Project\\Báo cáo tài chính Power BI\\BCTC\\db_file_financial_statement_analysis\\dim_hqhd.txt", index=False)
with open(r'D:\Tin hoc\Project\Báo cáo tài chính Power BI\BCTC\db_file_financial_statement_analysis\dim_hqhd.txt', newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE dim_hqhd')
    cursor.executemany(''' INSERT INTO dim_hqhd Values (?, ?, ?, ?, ?)''', reader)
    conn.commit()

In [42]:
dim_kctsnv["ETL_date"] = str(dt.datetime.now())

query = '''if (OBJECT_ID('dim_kctsnv') is not null)
begin
	print('abcd')
end
else
begin
	create table dim_kctsnv
	(
        [ID_1] varchar(10),
		[KM_1] nvarchar(100),
        [ID_2] varchar(10),
        [KM_2] nvarchar(100),
        [ETL_date] varchar(50)
	);
end
'''
cursor.execute(query)
conn.commit()

In [43]:
dim_kctsnv.to_csv(f"D:\\Tin hoc\\Project\\Báo cáo tài chính Power BI\\BCTC\\db_file_financial_statement_analysis\\dim_kctsnv.txt", index=False)
with open(r'D:\Tin hoc\Project\Báo cáo tài chính Power BI\BCTC\db_file_financial_statement_analysis\dim_kctsnv.txt', newline='', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    cursor.execute('TRUNCATE TABLE dim_kctsnv')
    cursor.executemany(''' INSERT INTO dim_kctsnv Values (?, ?, ?, ?, ?)''', reader)
    conn.commit()

In [44]:
df_fact_lctt.to_excel(r"D:\Tin hoc\Project\Báo cáo tài chính Power BI\BCTC\fact_cash_flow\fact_cash_flow.xlsx", index=False)
dim_cash_flow.to_excel(r"D:\Tin hoc\Project\Báo cáo tài chính Power BI\BCTC\dim_cash_flow\dim_cash_flow.xlsx", index=False)
dim_income_statement.to_excel(r"D:\Tin hoc\Project\Báo cáo tài chính Power BI\BCTC\dim_income_statement\dim_income_statement.xlsx", index=False)
df_fact_kqkd.to_excel(r"D:\Tin hoc\Project\Báo cáo tài chính Power BI\BCTC\fact_income_statement\fact_income_statement.xlsx", index=False)
df_dim_balance_sheet.to_excel(f"D:\\Tin hoc\\Project\\Báo cáo tài chính Power BI\\BCTC\\dim_balance_sheet\\dim_balance_sheet.xlsx", index=False)
fact_all.to_excel(f'D:\\Tin hoc\\Project\\Báo cáo tài chính Power BI\BCTC\\fact_balance_sheet\\fact_all.xlsx', index=False)

In [45]:
df_stock

,Stock,Phan_loai_1,Phan_loai_2,Image URL,ETL_date
0,BFC,Phi tài chính,Sản xuất hàng hoá,https://cdn.haitrieu.com/wp-content/uploads/20...,2025-09-07 14:51:43.018412
1,BSR,Phi tài chính,Sản xuất hàng hoá,https://bsr.com.vn/documents/37629/0/logo-BSR-...,2025-09-07 14:51:43.018412
2,DCM,Phi tài chính,Sản xuất hàng hoá,https://www.pvcfc.com.vn/Data/Sites/1/media/im...,2025-09-07 14:51:43.018412
3,DPM,Phi tài chính,Sản xuất hàng hoá,https://cafef1.mediacdn.vn/Images/Uploaded/DuL...,2025-09-07 14:51:43.018412
4,DPR,Phi tài chính,Sản xuất hàng hoá,https://cafef1.mediacdn.vn/LOGO/DPR.jpg,2025-09-07 14:51:43.018412
5,GMD,Phi tài chính,Sản xuất hàng hoá,https://cafef1.mediacdn.vn/LOGO/GMD.jpg,2025-09-07 14:51:43.018412
6,HAH,Phi tài chính,Sản xuất hàng hoá,https://cafef1.mediacdn.vn/LOGO/HAH.jpg,2025-09-07 14:51:43.018412
7,HPG,Phi tài chính,Sản xuất hàng hoá,https://sudospaces.com/hoaphat-com-vn/2019/10/...,2025-09-07 14:51:43.018412
8,IMP,Phi tài chính,Sản xuất hàng hoá,https://rs.nguoiquansat.vn/Image/Logo/IMP/logo...,2025-09-07 14:51:43.018412
9,NKG,Phi tài chính,Sản xuất hàng hoá,https://cafef1.mediacdn.vn/LOGO/NKG_1.PNG,2025-09-07 14:51:43.018412
